In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from sentence_transformers import SentenceTransformer

c:\Users\OLAMIDE\Desktop\Medical-Chatbot-Using-Llama2\mchatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "a226c353-7acd-4f5b-81d2-c21ce908167d"
PINECONE_API_ENV = "gcp-starter"

In [3]:
# Extract the data from the PDF
def load_pdf(data_directory):
    loader = DirectoryLoader(data_directory,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    
    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#extracted_data

In [6]:
# Split the text into smaller chunks
def text_splitter(extracted_data):
    text_split = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
    text_chunks = text_split.split_documents(extracted_data)
    
    return text_chunks

In [7]:
text_chunks = text_splitter(extracted_data)
print("Number of text chunks: ", len(text_chunks))

Number of text chunks:  7034


In [8]:
#text_chunks

In [9]:
# Load the embeddings
def load_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    return embeddings

In [10]:
embeddings = load_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result = embeddings.embed_query("What is the capital of France?")
print("Length:", len(query_result))

Length: 384


In [13]:
# query_result

In [14]:
from pinecone import Pinecone
pc = Pinecone(
api_key = PINECONE_API_KEY,    
)
pc.list_indexes().names() # to check if my index exsist
index_name = "medical-chatbot"
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.07034,
 'namespaces': {'medicalChatBot': {'vector_count': 7034}},
 'total_vector_count': 7034}

In [17]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    namespace="medicalChatBot",
    index_name='medical-chatbot'
)

vectorstore.add_texts(texts=[t.page_content for t in text_chunks])

['36835eca-aff7-402d-8190-63cd60cb3aa6',
 '6d54938d-0f25-4442-9191-a6bf589b33da',
 '140915a9-ff29-4e5a-9d41-019fe21aa80e',
 'ed945bea-8cd0-4760-b16b-1905e1d06962',
 'd44864bb-41cc-4a4e-b64a-bdd8bce03dde',
 'a2fea31c-7032-48cc-8d45-d3900b60b30b',
 'b6057958-70fb-4488-8d7c-8d1448cf94fe',
 'fa3a54fa-b27f-463a-9593-cf2af07c2fd3',
 '42e79beb-e990-43cc-9e11-826b74ccc836',
 '6f981d09-36ba-442d-ae2e-237bf510dd8c',
 '4ee9bbfc-6327-4f86-a98d-4017bc3577fb',
 '2f6075fb-c162-4d75-a932-58ee85c9809b',
 '44088ad5-29bf-47ea-bc74-2d8b606fd000',
 '42914c59-d740-4f34-a7bc-ea680dd5d167',
 'ae4a6170-212e-4b60-a4b4-023fe829472d',
 'ddf5d66f-6935-468d-bbdb-9b9de257c3bc',
 '22b8811b-11ec-4353-a7a8-940e4facc73f',
 'f5b4722a-288f-4ba3-84c6-39ee8001a9b6',
 '8510956c-5e63-42c6-af26-e5464d027c66',
 '12cbede0-615f-4961-a416-f868b5a64eda',
 'a6c169b5-aac9-4d06-be32-c7ab0cbab285',
 '3a13321b-57bf-4206-bd60-a8dda6f1254e',
 'f93a327a-f23a-4d6b-b3da-5c5b137c853c',
 '994bcd4d-1098-4a8b-8ca4-0930483366b0',
 '138b174d-2bcf-

In [18]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.07034,
 'namespaces': {'medicalChatBot': {'vector_count': 7034}},
 'total_vector_count': 7034}

In [19]:
def format_docs(docs):
    c = 0
    return "\n".join(doc.page_content for doc in docs)

In [20]:
query = "What is abortion?" 

doc_retrieved = vectorstore.similarity_search(query, k=3)

doc_retrieved_formatted = format_docs(doc_retrieved)
print("Type", type(doc_retrieved_formatted))
print("Length", len(doc_retrieved_formatted))
print(doc_retrieved_formatted)


Type <class 'str'>
Length 915
Abortion has been a legal procedure in the United Statessince 1973.
Purpose
An abortion may be performed whenever there is
some compelling reason to end a pregnancy. Womenhave abortions because continuing the pregnancy wouldcause them hardship, endanger their life or health, orbecause prenatal testing has shown that the fetus will beborn with severe abnormalities.
Abortions are safest when performed within the first
procedure to women seek-ing late-term abortion.
mother in carrying and giving birth to more than one ortwo babies, and also to decrease the risk of complicationsto the remaining fetus(es). The term selective abortionalso refers to choosing to abort a fetus for reasons such asthe woman is carrying a fetus which likely will be bornwith some birth defect or impairment, or because the sexof the fetus is not preferred by the individual.
Purpose
A woman may decide to abort for health reasons, for


In [21]:
prompt_template = """
Use the following information to answer the user question. 
If you need more information, please ask the user for clarification.
If you cannot find the answer, please let the user know, do not try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful Answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template,
                      input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [23]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={"max_new_tokens": 512,
                            "temperature": 0.8})

In [24]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    chain_type="stuff"
)

In [1]:
while True:
    user_input = input(f"Ask me a question: ")
    if user_input == "exit":
        break
    qa_response = qa.invoke({"query": user_input})
    print("Response:", qa_response["result"])

NameError: name 'qa' is not defined